In [ ]:
from dotenv import load_dotenv

load_dotenv("../../.env")

import dspy
import pickle
import time

from agentic_system.agents import CFEfficacyAgent
from agentic_system.litl_data.litl_utils import load_efficacy_devset

In [2]:
N_SAMPLE_REPEATS = 5

efficacy_devset = load_efficacy_devset()
efficacy_devset = efficacy_devset * N_SAMPLE_REPEATS

print(len(efficacy_devset))
efficacy_devset[:3]

210


[Example({'compound_name': 'Luminespib', 'cf_efficacy': 0.9384660045676458}) (input_keys={'compound_name'}),
 Example({'compound_name': 'Tanespimycin', 'cf_efficacy': 0.9303885695518936}) (input_keys={'compound_name'}),
 Example({'compound_name': 'MK-2866', 'cf_efficacy': 0.8625497422790986}) (input_keys={'compound_name'})]

In [ ]:
lm = dspy.LM(
    "gemini/gemini-2.5-pro",
    temperature=0.75,
    cache=False,
    max_tokens=80000,
    num_retries=10,
)

dspy.settings.configure(lm=lm, track_usage=True)
efficacy_agent = CFEfficacyAgent(use_litl=False)

In [4]:
NUM_THREADS = 20

start_time = time.time()
agent_runs = efficacy_agent.batch(efficacy_devset, num_threads=NUM_THREADS)
len(agent_runs)

Processed 18 / 210 examples:   9%|▊         | 18/210 [01:44<12:27,  3.89s/it]

2025/10/09 13:02:21 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=50000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Processed 56 / 210 examples:  27%|██▋       | 56/210 [03:54<08:57,  3.49s/it]

2025/10/09 13:04:30 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=50000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Processed 92 / 210 examples:  44%|████▍     | 92/210 [05:58<04:13,  2.15s/it]

...

2025/10/09 13:07:48 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=50000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Processed 188 / 210 examples:  90%|████████▉ | 188/210 [11:19<01:08,  3.11s/it]

2025/10/09 13:11:59 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=50000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Processed 210 / 210 examples: 100%|██████████| 210/210 [12:40<00:00,  3.62s/it]


210

In [ ]:
results = {}

name = "no_litl_runs"
cost = sum([x["cost"] for x in lm.history])
time_taken = time.time() - start_time

results["name"] = name
results["cost"] = cost
results["time"] = time_taken

results["agent_runs"] = []
for i, run in enumerate(agent_runs):
    results["agent_runs"].append((efficacy_devset[i], run))

In [ ]:
with open(f"./0.{name}.pkl", "wb") as f:
    pickle.dump(results, f)